# Oanda Demo Trading Notebook

## Packages & Tools

In [1]:
from utils.packages import *
from utils.tools import *

## API Setup

Read from config file

In [2]:
temp_file = 'config/access_keys.yaml'
with open(temp_file) as temp_file:
    config = yaml.load(temp_file)
    
temp_file = 'config/wma.yaml'
with open(temp_file) as temp_file:
    wma = yaml.load(temp_file)    

In [3]:
access_token = config['oanda_demo_account']['token']
accountID = config['oanda_demo_account']['account_id']
api = API(access_token = access_token)

instrument = wma['curr']['instrument']    
pip = wma['pip_size'][instrument]

short_moving_window = wma['wma']['short_moving_window']
long_moving_window = wma['wma']['long_moving_window']
lists_size = wma['wma']['lists_size']
slope_num = wma['wma']['slope_num']
min_order_angle = wma['wma']['min_order_angle']

units =  wma['order']['units']
profit_target =  wma['order']['profit_target']
loss_limit_num =  wma['order']['stop_loss_limit']
order_num_limit =  wma['order']['order_num_limit']
price_allowed_buffer =  wma['order']['price_allowed_buffer']

max_skips = wma['loss_skip']['max_skips']
consecutive_losses = wma['loss_skip']['consecutive_losses']

## Code Engine

In [4]:
params = {'instruments': instrument}
r = pricing.PricingStream(accountID=accountID, params=params)
rv = api.request(r)

skip_iter = 1
pl_pip = 0
long_wma = 0
short_wma = 0
closed_orders = 0
long_list_counter = 0


long_flag = True
short_flag = True
valid_iteration = True

old_direction =  'none'

list_ready = False
order_flag = False
negative_run =  False
first_position_changed = False
initial_position_captured = False

pl_list = collections.deque([])
tick_list = collections.deque([])
long_list = collections.deque([])
short_list = collections.deque([])
long_wma_list = collections.deque([])
short_wma_list = collections.deque([])

loss_limit = loss_limit_num * pip
profit_target = profit_target * pip 

long_weights = get_weights(long_moving_window)
short_weights = get_weights(short_moving_window)

In [5]:
for i, resp in tqdm(enumerate(rv)):        
    resp_type = resp['type']       

    if resp_type == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
        pass

    #-----------------------------------------------------------------------------------------------------
    elif resp_type == 'PRICE': # Check if we are yet to make the order               
        date_val, time_val, time_fraction = get_date_time(resp) # Get time stamp for reference            
        sell_price, buy_price, spread, tick_price = get_prices(resp) # Get prices from the response  

    #List Prep
    #-----------------------------------------------------------------------------------------------------
        #Short list prep
        #-----------------------------------------------------------------------------------------------------
        if len(short_list) < short_moving_window:
            short_list.append(tick_price)

        if short_flag and len(short_list) == short_moving_window:
            short_wma = sum(np.array(short_list)*short_weights)
            short_flag =False

        if short_flag == False and len(short_list) == short_moving_window:
            short_list.popleft()   
            short_list.append(tick_price) 
            short_wma = sum(np.array(short_list)*short_weights)



        #Long list prep   
        #-----------------------------------------------------------------------------------------------------                
        if len(long_list) < long_moving_window:
            long_list.append(tick_price)

        if long_flag and len(long_list) == long_moving_window:
            long_wma = sum(np.array(long_list)*long_weights)
            long_flag = False

        if long_flag == False and len(long_list) == long_moving_window:
            long_list.popleft()   
            long_list.append(tick_price) 
            long_wma = sum(np.array(long_list)*long_weights)   
            long_list_counter += 1
            if long_list_counter >= lists_size:
                list_ready = True
    #-----------------------------------------------------------------------------------------------------
  
        #Direction identification
        #----------------------------------------------------------------------------------------------------- 
        if valid_iteration == False and list_ready:
            #print(f'Gap: {max_gap_wma}, Threshold: {max_gap_wma_threshold}')
            pass
            
        elif valid_iteration and list_ready:            
            short_slope_x_plt, long_slope_x_plt, abline_short, abline_long, angle = get_slopes(short_wma_list, long_wma_list, slope_num, pip, lists_size)
            
            if list_ready:
                if short_wma > long_wma:
                    current_position = 'positive'
                elif short_wma < long_wma:
                    current_position = 'negative'
                elif short_wma == long_wma:
                    current_position = 'same level'

            if list_ready and initial_position_captured == False:
                initial_position = current_position
                print(f'initial_position : {initial_position}')
                initial_position_captured =  True
                winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC) 

            if list_ready and initial_position_captured:
                if initial_position == current_position:
                    pass

                elif current_position == 'same level':
                    print('Same level, wait for next iteration')

                elif initial_position != current_position and current_position != 'same level':
                    print(f'{initial_position} ---> {current_position}')

                    #Select order type -------------------------------------------------------------------------
                    if current_position == 'positive':
                        old_order_type = 'short'
                        order_type = 'long'
                        stop_price = buy_price - loss_limit                

                    elif current_position == 'negative':
                        old_order_type = 'long'
                        order_type = 'short'
                        stop_price = sell_price + loss_limit                

                    # Ordering -------------------------------------------------------------------------    
                    #-------------------------------------------------------------------------------------------                
                    if order_flag:
                        r = positions.OpenPositions(accountID=accountID)
                        if len(api.request(r)['positions']) > 0:
                            close_order_log = close_order(accountID, old_order_type ,instrument, api)
                            closed_orders += 1
                            print(f'Closed {old_order_type} position : {closed_orders}')
                            if closed_orders >= order_num_limit:
                                print(f'Closed {closed_orders} orders and exited')
                                break
                        else:
                            pass
                                
                        get_wma_output_data(accountID, instrument, 'direction_change', api, short_moving_window, long_moving_window, lists_size, units, loss_limit_num, profit_target, price_allowed_buffer, max_skips, consecutive_losses)
                        close_dict_key= list(close_order_log.keys())[1]
                        pl_pip = float(close_order_log[close_dict_key]['pl'])
                        order_flag = False

                        if len(pl_list) < consecutive_losses:
                            pl_list.append(pl_pip)
                        
                        if len(pl_list) == consecutive_losses:
                            negative_run = all(itrs < 0 for itrs in pl_list)
                            pl_list.popleft()
                            pl_list.append(pl_pip)

                            
                    if negative_run == False:
                        if angle >= min_order_angle:
                            print(f'Going {order_type}')
                            make_order_log = make_order(accountID, stop_price, instrument, units, order_type, api)
                            print('ordered')
                            first_long =  True
                            order_flag = True
                        else:
                            print(f'Angle:{angle} smaller than min_order_angle:{min_order_angle}')
                    else:
                        if max_skips > skip_iter:
                            print(f'Skipped iteration:{skip_iter}, pl_list : {pl_list}')
                            skip_iter += 1
                        elif max_skips <= skip_iter:
                            print(f'Skipped iteration:{skip_iter}, pl_list : {pl_list}')
                            negative_run = False
                            #pl_list = collections.deque([])
                            pl_list.popleft()
                            
                    initial_position = current_position

                    
                # Take Profit -------------------------------------------------------------------------    
                #-------------------------------------------------------------------------------------------                
                if initial_position == current_position and current_position != 'same level' and order_flag:
                    
                    if order_type == 'long':     
                        ordered_buy_price = float(make_order_log['orderFillTransaction']['fullPrice']['asks'][0]['price'])
                        profit = sell_price - ordered_buy_price 
                        
                        if first_long:
                            max_price = short_wma
                            first_long = False

                        max_price = max(short_wma, max_price)
                        buffered_max_price = max_price - (max_price * pip * price_allowed_buffer)

                        if profit >= profit_target and short_wma <= buffered_max_price:    
                            r = positions.OpenPositions(accountID=accountID)
                            if len(api.request(r)['positions']) > 0:
                                close_order_log = close_order(accountID, order_type ,instrument, api)   
                                closed_orders += 1
                                print(f'Closed {order_type} position : {closed_orders}')
                            else:
                                pass
                                
                            get_wma_output_data(accountID, instrument, 'Take_profit', api, short_moving_window, long_moving_window, lists_size, units, loss_limit_num, profit_target, price_allowed_buffer, max_skips, consecutive_losses)
                            order_flag = False
                            max_gap_wma = 0

                            
                    if order_type == 'short':     
                        ordered_sell_price = float(make_order_log['orderFillTransaction']['fullPrice']['bids'][0]['price'])
                        profit = ordered_sell_price - buy_price                
                        
                        if first_long:
                            min_price = short_wma
                            first_long = False

                        min_price = min(short_wma, min_price)
                        buffered_min_price = min_price + (min_price * pip * price_allowed_buffer)

                        if profit >= profit_target and short_wma >= buffered_min_price:
                            r = positions.OpenPositions(accountID=accountID)
                            if len(api.request(r)['positions']) > 0:
                                close_order_log = close_order(accountID, order_type ,instrument, api)     
                                closed_orders += 1
                                print(f'Closed {order_type} position : {closed_orders}')
                            else:
                                pass

                            get_wma_output_data(accountID, instrument, 'Take_profit', api, short_moving_window, long_moving_window, lists_size, units, loss_limit_num, profit_target, price_allowed_buffer, max_skips, consecutive_losses)
                            order_flag = False
                            max_gap_wma = 0             
                            
    if closed_orders >= order_num_limit:
        print(f'Closed {closed_orders} orders and exited')
        break

322it [10:22,  1.93s/it]


ValueError: Inputs must not be empty.

In [ ]:
df_copy = pd.read_csv('data/machine_use/wma_log_machine_use.csv')
df_copy.to_csv('data/wma_log.csv', index = False)